In [1]:
%pip install langchain langchain-aws langgraph


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langgraph.prebuilt import create_react_agent
from langgraph_checkpoint_aws import BedrockAgentCoreEventsSaver

In [3]:
from langgraph_checkpoint_aws.utils import generate_checkpoint_id

MEMORY_ID = "<memory-id>"
REGION = "us-east-1"
SESSION_ID = "cp_session1"
ACTOR_ID = generate_checkpoint_id("")

In [4]:
from langchain_core.tools import tool

@tool
def lookup_order(order_id: str) -> str:
    """Returns the order status"""
    return f"Customer order {order_id} was shipped 3 days ago, with an expected delivery tomorrow before 5pm"

@tool
def update_customer_email(email: str):
    """Updates customer's email address"""
    return f"Customer's email updated to {email}"

In [5]:
checkpointer = BedrockAgentCoreEventsSaver(
    memory_id=MEMORY_ID,
    region_name=REGION
)

In [6]:
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, SystemMessage

llm_model = "bedrock_converse:us.anthropic.claude-3-5-haiku-20241022-v1:0"

agent = create_react_agent(
    llm_model,
    tools=[lookup_order, update_customer_email],
    checkpointer=checkpointer
)

In [7]:
config = {
    "configurable": {
        "thread_id": SESSION_ID
    }
}

In [17]:
def list_events():
    params = {
        "memoryId": MEMORY_ID,
        "actorId": ACTOR_ID,
        "sessionId": SESSION_ID
    }
    response = checkpointer.client.list_events(**params)
    events = response.get("events", [])
    for event in events:
        print(f"\n================================ Event Id: {event['eventId']} =================================\n")
        for payload in event['payload']:
            import base64
            import json

            payload_blob = payload['blob']
            decoded_json = base64.b64decode(payload_blob).decode("utf-8")
            original_data = json.loads(decoded_json)
            print(f"================================ blob =================================")
            print(original_data)

In [9]:
response = agent.invoke({
    "messages": [
        "Hi, I'm having trouble with my order #12345"
    ]
}, config)

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi, I'm having trouble with my order #12345
================================== Ai Message ==================================

[{'type': 'text', 'text': "I'll help you check the status of your order. Let me look up the details for order #12345."}, {'type': 'tool_use', 'name': 'lookup_order', 'input': {'order_id': '12345'}, 'id': 'tooluse_beiJbnBRTK-2EsDOM3O6IQ'}]
Tool Calls:
  lookup_order (tooluse_beiJbnBRTK-2EsDOM3O6IQ)
 Call ID: tooluse_beiJbnBRTK-2EsDOM3O6IQ
  Args:
    order_id: 12345
================================= Tool Message =================================
Name: lookup_order

Customer order 12345 was shipped 3 days ago, with an expected delivery tomorrow before 5pm
================================== Ai Message ==================================

Based on the order lookup, your order #12345 was shipped 3 days ago and is expected to be delivered tomorrow before 5 PM. Is there anything specific a

In [21]:
list_events()


================================ Event Id: 0000001754081617000#57aeefbe =================================

================================ blob =================================
{'step_type': 'CHECKPOINTS', 'thread_id': 'cp_session1', 'checkpoint_ns': '', 'checkpoint': ['msgpack', 'hqF2A6J0c9kgMjAyNS0wOC0wMVQyMDo1MzozNy4xNzMwMDUrMDA6MDCiaWTZJDFmMDZmMTk5LTg3ZGMtNjYyNi04MDAzLWFiMGEzODExNWFjMK5jaGFubmVsX3ZhbHVlc4GobWVzc2FnZXOUx98FlL1sYW5nY2hhaW5fY29yZS5tZXNzYWdlcy5odW1hbqxIdW1hbk1lc3NhZ2WHp2NvbnRlbnTZK0hpLCBJJ20gaGF2aW5nIHRyb3VibGUgd2l0aCBteSBvcmRlciAjMTIzNDWxYWRkaXRpb25hbF9rd2FyZ3OAsXJlc3BvbnNlX21ldGFkYXRhgKR0eXBlpWh1bWFupG5hbWXAomlk2SRiMDYzM2QzYy0zZDE4LTRlYjQtYjZmMy0zNDNlOWI1ODYwMTinZXhhbXBsZcKzbW9kZWxfdmFsaWRhdGVfanNvbsgD0gWUumxhbmdjaGFpbl9jb3JlLm1lc3NhZ2VzLmFpqUFJTWVzc2FnZYqnY29udGVudJKCpHR5cGWkdGV4dKR0ZXh02VpJJ2xsIGhlbHAgeW91IGNoZWNrIHRoZSBzdGF0dXMgb2YgeW91ciBvcmRlci4gTGV0IG1lIGxvb2sgdXAgdGhlIGRldGFpbHMgZm9yIG9yZGVyICMxMjM0NS6EpHR5cGWodG9vbF91c2WkbmFtZaxsb29rdXBfb3JkZXKlaW5wdXSBqG9

In [22]:
response = agent.invoke({
    "messages": [
        "Actually, before that - I also want to change my email address"
    ]
}, config)
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi, I'm having trouble with my order #12345
================================== Ai Message ==================================

[{'type': 'text', 'text': "I'll help you check the status of your order. Let me look up the details for order #12345."}, {'type': 'tool_use', 'name': 'lookup_order', 'input': {'order_id': '12345'}, 'id': 'tooluse_beiJbnBRTK-2EsDOM3O6IQ'}]
Tool Calls:
  lookup_order (tooluse_beiJbnBRTK-2EsDOM3O6IQ)
 Call ID: tooluse_beiJbnBRTK-2EsDOM3O6IQ
  Args:
    order_id: 12345
================================= Tool Message =================================
Name: lookup_order

Customer order 12345 was shipped 3 days ago, with an expected delivery tomorrow before 5pm
================================== Ai Message ==================================

Based on the order lookup, your order #12345 was shipped 3 days ago and is expected to be delivered tomorrow before 5 PM. Is there anything specific a

In [23]:
list_events()


================================ Event Id: 0000001754082838000#6598bb19 =================================

================================ blob =================================
{'step_type': 'CHECKPOINTS', 'thread_id': 'cp_session1', 'checkpoint_ns': '', 'checkpoint': ['msgpack', 'hqF2A6J0c9kgMjAyNS0wOC0wMVQyMToxMzo1OC4yOTYwMDgrMDA6MDCiaWTZJDFmMDZmMWM3LTA1NjUtNmFmYy04MDA2LTU0OTIyMzgyNDkwNK5jaGFubmVsX3ZhbHVlc4GobWVzc2FnZXOWx98FlL1sYW5nY2hhaW5fY29yZS5tZXNzYWdlcy5odW1hbqxIdW1hbk1lc3NhZ2WHp2NvbnRlbnTZK0hpLCBJJ20gaGF2aW5nIHRyb3VibGUgd2l0aCBteSBvcmRlciAjMTIzNDWxYWRkaXRpb25hbF9rd2FyZ3OAsXJlc3BvbnNlX21ldGFkYXRhgKR0eXBlpWh1bWFupG5hbWXAomlk2SRiMDYzM2QzYy0zZDE4LTRlYjQtYjZmMy0zNDNlOWI1ODYwMTinZXhhbXBsZcKzbW9kZWxfdmFsaWRhdGVfanNvbsgD0gWUumxhbmdjaGFpbl9jb3JlLm1lc3NhZ2VzLmFpqUFJTWVzc2FnZYqnY29udGVudJKCpHR5cGWkdGV4dKR0ZXh02VpJJ2xsIGhlbHAgeW91IGNoZWNrIHRoZSBzdGF0dXMgb2YgeW91ciBvcmRlci4gTGV0IG1lIGxvb2sgdXAgdGhlIGRldGFpbHMgZm9yIG9yZGVyICMxMjM0NS6EpHR5cGWodG9vbF91c2WkbmFtZaxsb29rdXBfb3JkZXKlaW5wdXSBqG9

In [24]:
response = agent.invoke({
    "messages": [
        "myemail@example.com"
    ]
}, config)
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi, I'm having trouble with my order #12345
================================== Ai Message ==================================

[{'type': 'text', 'text': "I'll help you check the status of your order. Let me look up the details for order #12345."}, {'type': 'tool_use', 'name': 'lookup_order', 'input': {'order_id': '12345'}, 'id': 'tooluse_beiJbnBRTK-2EsDOM3O6IQ'}]
Tool Calls:
  lookup_order (tooluse_beiJbnBRTK-2EsDOM3O6IQ)
 Call ID: tooluse_beiJbnBRTK-2EsDOM3O6IQ
  Args:
    order_id: 12345
================================= Tool Message =================================
Name: lookup_order

Customer order 12345 was shipped 3 days ago, with an expected delivery tomorrow before 5pm
================================== Ai Message ==================================

Based on the order lookup, your order #12345 was shipped 3 days ago and is expected to be delivered tomorrow before 5 PM. Is there anything specific a

In [25]:
list_events()


================================ Event Id: 0000001754082948000#8d5f7411 =================================

================================ blob =================================
{'step_type': 'CHECKPOINTS', 'thread_id': 'cp_session1', 'checkpoint_ns': '', 'checkpoint': ['msgpack', 'hqF2A6J0c9kgMjAyNS0wOC0wMVQyMToxNTo0OC44NzU0MDIrMDA6MDCiaWTZJDFmMDZmMWNiLTIzZjYtNmVmZS04MDBiLTAyN2U3YTY3MjUxZK5jaGFubmVsX3ZhbHVlc4GobWVzc2FnZXOax98FlL1sYW5nY2hhaW5fY29yZS5tZXNzYWdlcy5odW1hbqxIdW1hbk1lc3NhZ2WHp2NvbnRlbnTZK0hpLCBJJ20gaGF2aW5nIHRyb3VibGUgd2l0aCBteSBvcmRlciAjMTIzNDWxYWRkaXRpb25hbF9rd2FyZ3OAsXJlc3BvbnNlX21ldGFkYXRhgKR0eXBlpWh1bWFupG5hbWXAomlk2SRiMDYzM2QzYy0zZDE4LTRlYjQtYjZmMy0zNDNlOWI1ODYwMTinZXhhbXBsZcKzbW9kZWxfdmFsaWRhdGVfanNvbsgD0gWUumxhbmdjaGFpbl9jb3JlLm1lc3NhZ2VzLmFpqUFJTWVzc2FnZYqnY29udGVudJKCpHR5cGWkdGV4dKR0ZXh02VpJJ2xsIGhlbHAgeW91IGNoZWNrIHRoZSBzdGF0dXMgb2YgeW91ciBvcmRlci4gTGV0IG1lIGxvb2sgdXAgdGhlIGRldGFpbHMgZm9yIG9yZGVyICMxMjM0NS6EpHR5cGWodG9vbF91c2WkbmFtZaxsb29rdXBfb3JkZXKlaW5wdXSBqG9

In [26]:
response = agent.invoke({
    "messages": [
        "The package already arrived, but seems damaged"
    ]
}, config)
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi, I'm having trouble with my order #12345
================================== Ai Message ==================================

[{'type': 'text', 'text': "I'll help you check the status of your order. Let me look up the details for order #12345."}, {'type': 'tool_use', 'name': 'lookup_order', 'input': {'order_id': '12345'}, 'id': 'tooluse_beiJbnBRTK-2EsDOM3O6IQ'}]
Tool Calls:
  lookup_order (tooluse_beiJbnBRTK-2EsDOM3O6IQ)
 Call ID: tooluse_beiJbnBRTK-2EsDOM3O6IQ
  Args:
    order_id: 12345
================================= Tool Message =================================
Name: lookup_order

Customer order 12345 was shipped 3 days ago, with an expected delivery tomorrow before 5pm
================================== Ai Message ==================================

Based on the order lookup, your order #12345 was shipped 3 days ago and is expected to be delivered tomorrow before 5 PM. Is there anything specific a

In [27]:
list_events()


================================ Event Id: 0000001754083009000#946b2598 =================================

================================ blob =================================
{'step_type': 'CHECKPOINTS', 'thread_id': 'cp_session1', 'checkpoint_ns': '', 'checkpoint': ['msgpack', 'hqF2A6J0c9kgMjAyNS0wOC0wMVQyMToxNjo0OS40NDE3OTIrMDA6MDCiaWTZJDFmMDZmMWNkLTY1OTItNjEwNC04MDBlLTA2NmY0NDk5Mjg3Za5jaGFubmVsX3ZhbHVlc4GobWVzc2FnZXOcx98FlL1sYW5nY2hhaW5fY29yZS5tZXNzYWdlcy5odW1hbqxIdW1hbk1lc3NhZ2WHp2NvbnRlbnTZK0hpLCBJJ20gaGF2aW5nIHRyb3VibGUgd2l0aCBteSBvcmRlciAjMTIzNDWxYWRkaXRpb25hbF9rd2FyZ3OAsXJlc3BvbnNlX21ldGFkYXRhgKR0eXBlpWh1bWFupG5hbWXAomlk2SRiMDYzM2QzYy0zZDE4LTRlYjQtYjZmMy0zNDNlOWI1ODYwMTinZXhhbXBsZcKzbW9kZWxfdmFsaWRhdGVfanNvbsgD0gWUumxhbmdjaGFpbl9jb3JlLm1lc3NhZ2VzLmFpqUFJTWVzc2FnZYqnY29udGVudJKCpHR5cGWkdGV4dKR0ZXh02VpJJ2xsIGhlbHAgeW91IGNoZWNrIHRoZSBzdGF0dXMgb2YgeW91ciBvcmRlci4gTGV0IG1lIGxvb2sgdXAgdGhlIGRldGFpbHMgZm9yIG9yZGVyICMxMjM0NS6EpHR5cGWodG9vbF91c2WkbmFtZaxsb29rdXBfb3JkZXKlaW5wdXSBqG9